# Neural Network creation

This script takes results from "aggregated results", and generates neural
networks based on them.


In [ ]:
import sys
from utils.setup import SetupNeuralNetworks

argv           = sys.argv[1:]
# argv           = ['-c', 'cfg_single_lon120.yml']
argv = ["-c", "cfg_testing.yml"]

setup = SetupNeuralNetworks(argv)


# Create Neural Networks

Rasp et al.'s CBRAIN `fc_model` is used here for reference. As we are
doing a simpler version, I've opted to create our own, shorter function
for NN creation.

## List of models

In [ ]:
from neural_networks.models import generate_models
model_descriptions = generate_models(setup)

## Read aggregated results and generate CausalSingleNN models

Currently, results are loaded from a previously created file. This was
made for testing purposes, and we may want to replace it for an analysis.

In that replacement, the idea would be to execute more or less the same
as aggregate_results up to the moment the file is currently saved, and
continue with that object (aggregated results) here.

## Generate SingleNN models

## Generate CausalSingleNN models

# Training

## Data generator

## Fit models

Train all models in the model list and store the results.

In [ ]:
from neural_networks.training import train_all_models

train_all_models(model_descriptions, setup)